In [ ]:
import pandas as pd
import xarray as xr
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from shapely.geometry import Point
import plotly.express as px
import datetime as dt

In [ ]:
### Functions to extract data from netcdf files


def get_variable_from_netcfd(df, lon, lat, var):
    """Extract variable from netcdf file and interpolate to 30 min intervals"""
    return (
        df[var]
        .sel(longitude=lon, latitude=lat, method="nearest")
        .to_dataframe()
        .set_index("valid_time")[[var]]
        .resample("1800s")
        .interpolate()
    )


def get_ssrd_from_netcfd(df, lon, lat):
    _ssrd = get_variable_from_netcfd(df, lon, lat, "ssrd")
    _ssrd["radiation"] = (_ssrd["ssrd"] - _ssrd["ssrd"].shift(1)).clip(lower=0)
    _ssrd["radiation"] = _ssrd["radiation"].fillna(0)
    return _ssrd[["radiation"]]

In [ ]:
# Load training data

training = pd.read_csv("data/training_data.csv")[["dtm", "solar_generation_MW"]]
training["valid_time"] = pd.to_datetime(training["dtm"])
training = training.drop(columns=["dtm"])

In [ ]:
# Load weather data

nwp = xr.load_dataset("data/hres_1day_south_scotland_202101_202306.nc")

In [ ]:
solar_variables = ["t2m", "d2m", "lcc", "mcc", "hcc", "tp"]

In [ ]:
# Create range of 4 latitudes and 4 longitudes
# from the maximum and minimum values of the NWP data

lat_range = np.linspace(
    nwp.latitude.values.tolist()[0], nwp.latitude.values.tolist()[-1], 4
)
lon_range = np.linspace(nwp.longitude.min(), nwp.longitude.max(), 4)

lat_range = nwp.latitude.values.tolist()
lon_range = nwp.longitude.values.tolist()

In [ ]:
scotland_gdf = gpd.read_file("lad.json")

In [ ]:
cross_array = np.array(np.meshgrid(lat_range, lon_range)).T.reshape(-1, 2)
points = [Point(lon, lat) for lat, lon in cross_array]
# Inside Merge
inside_points = [point for point in points if scotland_gdf.contains(point).any()]
inside_array = np.array([[point.y, point.x] for point in inside_points])

In [ ]:
ssrd = pd.DataFrame()

for lat, lon in inside_array:
    weather = get_ssrd_from_netcfd(nwp, lon, lat)
    weather["latitude"] = round(lat, 1)
    weather["longitude"] = round(lon, 2)
    ssrd = pd.concat([ssrd, weather])

ssrd = ssrd.reset_index()
ssrd["valid_time"] = ssrd["valid_time"].dt.tz_localize("UTC")

In [ ]:
# temperature = pd.DataFrame()
# for lat, lon in inside_array:
#     weather = get_variable_from_netcfd(nwp, lon, lat, "t2m")
#     weather["latitude"] = round(lat, 1)
#     weather["longitude"] = round(lon, 2)
#     temperature = pd.concat([temperature, weather])

# temperature = temperature.reset_index()
# temperature["valid_time"] = temperature["valid_time"].dt.tz_localize("UTC")

In [ ]:
cloud = pd.DataFrame()
for lat, lon in inside_array:
    weather = get_variable_from_netcfd(nwp, lon, lat, "lcc")
    weather["latitude"] = round(lat, 1)
    weather["longitude"] = round(lon, 2)
    cloud = pd.concat([cloud, weather])

cloud = cloud.reset_index()
cloud["valid_time"] = cloud["valid_time"].dt.tz_localize("UTC")

In [ ]:
# lcc = cloud.groupby("valid_time")['lcc'].mean().reset_index()

In [ ]:
all_merge_radiation = ssrd.merge(training, on="valid_time", how="left")

In [ ]:
training_radiation = all_merge_radiation[all_merge_radiation["solar_generation_MW"].notnull()]

In [ ]:
submission_radiation = all_merge_radiation[all_merge_radiation["solar_generation_MW"].isnull()]

In [ ]:
# training_radiation.sample(1000).plot(
#     x="radiation", y="solar_generation_MW", kind="scatter"
# )

In [ ]:
correlations = (
    pd.DataFrame(
        training_radiation.groupby(by=["latitude", "longitude"])[
            ["solar_generation_MW", "radiation"]
        ]
        .corr(method = 'pearson')
        .iloc[0::2, -1]
    )
    .reset_index()
    .drop(columns=["level_2"])
    .rename(columns={"radiation": "correlation"})
)

correlations["correlation"].hist(bins=200)

In [ ]:
latitudes = nwp.latitude.values.tolist()
longitudes = nwp.longitude.values.tolist()

fig, ax = plt.subplots()

scotland_gpd_df = gpd.read_file("lad.json")
scotland_plot = scotland_gpd_df.plot(ax=ax)
scotland_plot.set_xlim(-8, -1.5)
scotland_plot.set_ylim(54, 60)


rect = Rectangle(
    (nwp.longitude.min() - 0.05, nwp.latitude.min() - 0.05),
    (nwp.longitude.max() - nwp.longitude.min()) + 0.1,
    nwp.latitude.max() - nwp.latitude.min() + 0.1,
    linewidth=1,
    edgecolor="r",
    facecolor="none",
)

scotland_plot.scatter(
    correlations["longitude"],
    correlations["latitude"],
    c=correlations["correlation"],
    s=50,
    cmap="coolwarm",
)


fig.set_size_inches(10, 10)

ax.set_ylim(54.5, 56.6)
ax.set_xlim(-6, -1.8)

ax.add_patch(rect)

plt.show()

In [ ]:
# ### Using one location
# select_location = (
#     correlations.sort_values(by="correlation", ascending=False)
#     .head(1)
#     .reset_index(drop=True)
# )

# select_latitude = select_location["latitude"][0]
# select_longitude = select_location["longitude"][0]

# select_location_df = training_radiation[
#     (training_radiation["latitude"] == select_latitude)
#     & (training_radiation["longitude"] == select_longitude)
# ].reset_index(drop=True)

In [ ]:
### Using a few locations with higher correlation

# Fraction = 10%

select_locations = (
    correlations.sort_values(by="correlation", ascending=False)
    .head(int(len(correlations) * 0.5))
    .reset_index(drop=True)
)

# Filter by selection locations
_select_location_df = (
    training_radiation.merge(
        select_locations, on=["latitude", "longitude"], how="inner"
    )
    # .merge(temperature, on=["latitude", "longitude", "valid_time"], how="inner")
    .groupby(by=["valid_time", "latitude", "longitude"])
    .mean()
    .reset_index()
)

select_location_df = _select_location_df.groupby(by=["valid_time"]).mean().reset_index()[['valid_time','radiation','solar_generation_MW']]

In [ ]:
submission_location_df = (
    submission_radiation.merge(
        select_locations, on=["latitude", "longitude"], how="inner"
    )
    .groupby(by=["valid_time", "latitude", "longitude"])
    .mean()
    .reset_index()
).drop(columns=["solar_generation_MW",'correlation'])

In [ ]:
# Merge average cloud cover
# select_location_df = select_location_df.merge(lcc, on=['valid_time'], how='inner')

In [ ]:
# ## Addition features

# select_location_df["days_since_start_of_year"] = select_location_df[
#     "valid_time"
# ].dt.dayofyear
# select_location_df["half_hour"] = (
#     select_location_df["valid_time"].dt.hour * 2
#     + select_location_df["valid_time"].dt.minute / 30
# )


# select_location_df["sin_days"] = np.sin(
#     2 * np.pi * select_location_df["days_since_start_of_year"] / 365
# )
# select_location_df["cos_days"] = np.cos(
#     2 * np.pi * select_location_df["days_since_start_of_year"] / 365
# )
# select_location_df["sin_hh"] = np.sin(2 * np.pi * select_location_df["half_hour"] / 48)
# select_location_df["cos_hh"] = np.cos(2 * np.pi * select_location_df["half_hour"] / 48)

In [ ]:
from sklearn.model_selection import train_test_split, KFold
from sklearn.linear_model import (
    LinearRegression,
    Ridge,
    Lasso,
    ElasticNet,
    BayesianRidge,
    HuberRegressor,
)



from sklearn.svm import SVR
from sklearn.ensemble import (
    RandomForestRegressor,
    GradientBoostingRegressor,
    AdaBoostRegressor,
)
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import TimeSeriesSplit

In [ ]:
for df in [select_location_df, submission_location_df]:
    # Adding lagged features
    df["radiation_lag_-1"] = df["radiation"].shift(-1)
    df["radiation_lag_1"] =  df["radiation"].shift(1)
    df["radiation_lag_2"] =  df["radiation"].shift(2)
    df["radiation_lag_-2"] = df["radiation"].shift(-2)
    df["radiation_lag_3"] =  df["radiation"].shift(3)
    df["radiation_lag_-3"] = df["radiation"].shift(-3)
    df["radiation_lag_4"] =  df["radiation"].shift(4)
    df["radiation_lag_-4"] = df["radiation"].shift(-4)
    df["radiation_lag_5"] =  df["radiation"].shift(5)
    df["radiation_lag_-5"] = df["radiation"].shift(-5)
    df["radiation_lag_6"] =  df["radiation"].shift(6)
    df["radiation_lag_-6"] = df["radiation"].shift(-6)

select_location_df = select_location_df.fillna(0)
submission_location_df = submission_location_df.fillna(0)

In [ ]:
selected_features = [
    "radiation",
    # "t2m",
    # "days_since_start_of_year",
    # "half_hour",
    # "sin_days",
    # "cos_days",
    # "sin_hh",
    # "cos_hh",
    # "lcc",

    "radiation_lag_1",
    "radiation_lag_-1",

    # # "lcc_lag_1",
    # # "lcc_lag_-1",    
    "radiation_lag_3",
    "radiation_lag_-3",

    "radiation_lag_5",
    "radiation_lag_-5",
]

In [ ]:
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# X_pre = select_location_df[selected_features].fillna(0)
# X = scaler.fit_transform(X_pre) 
# y = select_location_df["solar_generation_MW"]

X = select_location_df[selected_features].fillna(0)
y = select_location_df["solar_generation_MW"]

In [ ]:
lr_pipeline = LinearRegression()
ridge_pipeline = Ridge()
lasso_pipeline = Lasso()
elastic_pipeline = ElasticNet()
bayesian_pipeline = BayesianRidge()
huber_pipeline = HuberRegressor()

In [ ]:
tss = TimeSeriesSplit(n_splits=6)
kf = KFold(n_splits=5, shuffle=True)
# for train_index, test_index in tss.split(X):

model_dict = {}

set_index = 0 
for idx, model in enumerate([
    lr_pipeline,
    # ridge_pipeline,
    huber_pipeline,]):
    mae = []
    
    indexes = {}
    train_index_counter = 0
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        indexes[train_index_counter] = train_index



        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        y_pred = np.array(y_pred)
        y_pred[y_pred < 0] = 0

        mae.append(mean_absolute_error(y_test, y_pred))
        train_index_counter += 1

    model_dict[idx] = {
        "model": model,
        "mae": round(np.mean(mae),2),
        # location of minimum mae in the list of mae
        "best": np.argmin(mae),
        "best_index": indexes[np.argmin(mae)],

    }
    set_index += 1
    print(round(np.mean(mae),2))

In [ ]:
chosen_model = model_dict[1]["model"]
train_index = model_dict[1]["best_index"]

chosen_train_data_X = X.iloc[train_index]
chosen_train_data_y = y.iloc[train_index]


In [ ]:
chosen_model.fit(chosen_train_data_X, chosen_train_data_y)

In [ ]:
select_location_df['lr_prediction'] = lr_pipeline.predict(select_location_df[selected_features])
select_location_df['lr_prediction'] = select_location_df['lr_prediction'].clip(lower=0)
# select_location_df['ridge_prediction'] = ridge_pipeline.predict(select_location_df[selected_features])
# select_location_df['ridge_prediction'] = select_location_df['ridge_prediction'].clip(lower=0)
select_location_df['huber_prediction'] = huber_pipeline.predict(select_location_df[selected_features])
select_location_df['huber_prediction'] = select_location_df['huber_prediction'].clip(lower=0)

In [ ]:
select_location_df.huber_prediction.sum()/select_location_df.solar_generation_MW.sum()

In [ ]:
select_location_df.head(20590).tail(50)[['solar_generation_MW','huber_prediction']]

In [ ]:
uncertainty = select_location_df[['valid_time', 'solar_generation_MW','huber_prediction','radiation']]

In [ ]:
uncertainty['date'] = uncertainty['valid_time'].dt.date
uncertainty['delta'] = uncertainty['solar_generation_MW'] - uncertainty['huber_prediction']
uncertainty['error'] = 100 * ( uncertainty['delta'] / uncertainty['solar_generation_MW'])

In [ ]:
uncertainty.groupby(by = ['date'])['delta'].sum().reset_index().sort_values('delta')

In [ ]:
uncertainty['days_since_start_of_year'] = pd.to_datetime(uncertainty['date']).dt.dayofyear
uncertainty['month'] = pd.to_datetime(uncertainty['date']).dt.month
uncertainty['cos_days'] = np.sin(2 * np.pi * uncertainty["days_since_start_of_year"] / 365)

In [ ]:
fig = px.scatter(uncertainty.sample(10000), x='radiation', y = 'delta', color='month', color_continuous_scale=px.colors.cyclical.Twilight)
fig.update_layout(
    title="Forecast Delta vs Forecast Irradiance",
    xaxis_title="Irradiance",
    yaxis_title="Delta - MW",
    legend_title="Month",
    
)
# x axis range
fig.update_yaxes(range=[-140, 140])
fig.update_layout(width = 1000, height = 600)

In [ ]:
fig = px.scatter(uncertainty.sample(10000).query("error < 100"), x='radiation', y = 'error', color='month', color_continuous_scale=px.colors.cyclical.Twilight)
fig.update_layout(
    title="Forecast Error vs Forecast Irradiance",
    xaxis_title="Irradiance",
    yaxis_title="Error - %",
    legend_title="Month",
    
)
# x axis range
fig.update_yaxes(range=[-140, 140])
fig.update_layout(width = 1000, height = 600)

In [ ]:
day_number = (dt.date(2023,3,6) - dt.date(2022,1,1)).days

In [ ]:
nwp['cc'] = nwp['lcc'] + nwp['mcc'] + nwp['hcc']

In [ ]:
# Plot for Gif

for n in range(1, 24):
    (nwp["ssrd"].isel(time=day_number, step=n) - nwp["ssrd"].isel(time=day_number, step=0)).plot() 
    plt.savefig(f"{n}_pic.png")
    plt.clf()

for n in range(1, 24):
    (nwp["cc"].isel(time=day_number, step=n) - nwp["ssrd"].isel(time=day_number, step=0)).plot() 
    plt.savefig(f"{n}_pic.png")
    plt.clf()
    

In [ ]:
fig = px.scatter(uncertainty.sample(10000).query("error < 100"), x='solar_generation_MW', y = 'error', color='month', color_continuous_scale=px.colors.cyclical.Twilight)
fig.update_layout(
    title="Forecast Error vs Forecast Irradiance",
    xaxis_title="Irradiance",
    yaxis_title="Error - %",
    legend_title="Month",
    
)
# x axis range
fig.update_yaxes(range=[-140, 140])
fig.update_layout(width = 1000, height = 600)

In [ ]:
fig = px.line(select_location_df, x='valid_time', y=['huber_prediction','solar_generation_MW'])
        
fig.update_traces(opacity=.4)

In [ ]:
submission = submission_location_df[['valid_time']].drop_duplicates().reset_index(drop = True)
submission_location_df_X = submission_location_df.drop(columns = ['latitude','longitude']).groupby('valid_time').mean()[selected_features]
submission['huber_prediction'] = chosen_model.predict(submission_location_df_X)
submission['huber_prediction'] = submission['huber_prediction'].clip(lower=0)

In [ ]:
submission.head(8000).tail(100).plot(x='valid_time', y=['huber_prediction'], kind='line')

In [ ]:
final_submission = pd.read_csv('data/submission_data_[TEAM_NAME].csv')

In [ ]:
final_submission['solar_generation_MW'] = submission['huber_prediction']
final_submission['solar_generation_MW'] = final_submission['solar_generation_MW'].ffill()

In [ ]:
final_submission.to_csv('bf_solar_forecast.csv', index=False)